In [1]:
import os
import numpy
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import keras

/dvlscratch/SHI/users/hofman/ML/venv3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [2]:
STA = 'PPT'

In [3]:
#load data from oracle do pandas
#connect to udb
with open(os.path.join('/','home','hofman','.dbp.txt'), 'r') as f: password_old = f.read().strip()
import cx_Oracle
connection = cx_Oracle.connect('hofman', password_old, 'udb')

In [4]:
from datetime import datetime as dt
ts_2017 = dt(2017,1,1,0,0,0).timestamp()
ts_2018 = dt(2018,1,1,0,0,0).timestamp()
ts_2019 = dt(2019,1,1,0,0,0).timestamp()

In [5]:
query = f"select * from EVAL_QANDEF_PPT"   ## where sta='{STA}'"
df_all_ = pd.read_sql(query, con=connection)

In [6]:
df_all_.shape

(25506, 12)

In [7]:
ind = [{False: True, True:False}[x] for x in numpy.isnan(df_all_['NCLS_PHASE_ODB'])]

In [8]:
ind

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [9]:
df_all = df_all_[ind]

In [10]:
df_all.head()

,ARID_ODB,TIME_ODB,AZI_ODB,IPHASE_QANDEF,IPHASE_ODB,ASSOC_PHASE,CLASS_IPHASE_QANDEF,CLASS_IPHASE_ODB,CLASS_PHASE_ODB,NCLS_IPHASE_QANDEF,NCLS_IPHASE_ODB,NCLS_PHASE_ODB
0,128815680,1.514767e+09,256.784330,N,N,None,N,N,N,0,0,0
1,128816457,1.514769e+09,8.829138,N,N,None,N,N,N,0,0,0
2,128817094,1.514772e+09,144.814780,N,N,None,N,N,N,0,0,0
3,128817389,1.514773e+09,249.318170,P,Pn,None,T,regP,N,1,2,0
4,128817390,1.514773e+09,178.937790,N,N,None,N,N,N,0,0,0


In [11]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    NCLS_IPHASE_QANDEF = df_all['NCLS_IPHASE_QANDEF'].as_matrix()
    NCLS_IPHASE_ODB = df_all['NCLS_IPHASE_ODB'].as_matrix()
    NCLS_PHASE_ODB = numpy.array(df_all['NCLS_PHASE_ODB'].as_matrix(), dtype=numpy.int)

In [12]:
NCLS_PHASE_ODB.min(), NCLS_PHASE_ODB.max()

(0, 1)

In [13]:
df_all[numpy.isnan(df_all['NCLS_PHASE_ODB'])]


,ARID_ODB,TIME_ODB,AZI_ODB,IPHASE_QANDEF,IPHASE_ODB,ASSOC_PHASE,CLASS_IPHASE_QANDEF,CLASS_IPHASE_ODB,CLASS_PHASE_ODB,NCLS_IPHASE_QANDEF,NCLS_IPHASE_ODB,NCLS_PHASE_ODB


In [14]:
from sklearn.metrics import confusion_matrix

In [15]:
def confmat(x,y):
    C = confusion_matrix(x,y)
    diagsum = numpy.diag(C).sum()
    accuracy = diagsum/C.sum()
    print(C)
    print(C.sum(axis=0), C.sum()) 
    AACC = (numpy.diag(C)[1:].sum())/(C[:,1:].sum())*100
    ACC = (accuracy*100)
    N = (C[0,1:].sum()/C[:,1:].sum()*100)
    print('Overal accuracy: %3.1f%%' % ACC)
    print('Assoc accuracy: %3.1f%%' % AACC)
    print('N phase rate: %3.1f%%' % N)
    return ACC, AACC, N

In [16]:
#confusion_matrix(y_true, y_pred)
print('OLD:')
ACC1, AACC1, N1 = confmat(NCLS_IPHASE_ODB, NCLS_PHASE_ODB)

print('NEW')
ACC2, AACC2, N2 = confmat(NCLS_IPHASE_QANDEF, NCLS_PHASE_ODB)

print('Difference:')
print("%3.1f%%" % (ACC2-ACC1), " %3.1f%%" % (AACC2-AACC1), " %3.1f%%" % (N2-N1))

OLD:
[[14524    53     0     0]
 [ 3937   196     0     0]
 [ 1469    68     0     0]
 [ 5245    14     0     0]]
[25175   331     0     0] 25506
Overal accuracy: 57.7%
Assoc accuracy: 59.2%
N phase rate: 16.0%
NEW
[[19734    36     0     0]
 [ 4935   230     0     0]
 [  150    64     0     0]
 [  356     1     0     0]]
[25175   331     0     0] 25506
Overal accuracy: 78.3%
Assoc accuracy: 69.5%
N phase rate: 10.9%
Difference:
20.6%  10.3%  -5.1%
